In [1]:
%load_ext rustdef

load rustdef


<IPython.core.display.Javascript object>

In [2]:
%rustdef depends --help

usage: %rustdef depends [-h] crates [crates ...]

Add crate dependencies

positional arguments:
  crates      Dependencies to be added

optional arguments:
  -h, --help  show this help message and exit


In [3]:
%rustdef depends ndarray numpy

In [4]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// immutable example
fn axpy_rs(a: f64, x: ArrayViewD<'_, f64>, y: ArrayViewD<'_, f64>) -> ArrayD<f64> {
    a * &x + &y
}

#[pyfn(m, "axpy")]
fn axpy<'py>(
    py: Python<'py>,
    a: f64,
    x: PyReadonlyArrayDyn<'py, f64>,
    y: PyReadonlyArrayDyn<'py, f64>,
) -> &'py PyArrayDyn<f64> {
    let x = x.as_array();
    let y = y.as_array();
    axpy_rs(a, x, y).into_pyarray(py)
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.6m at /home/ubuntu/.cache/pypoetry/virtualenvs/rustdef-WQU2FD6R-py3.6/bin/python
   Compiling pyo3 v0.13.0
   Compiling numpy v0.13.0=====================================>    ] 45/48/48
   Compiling rustdef_cell_061582ff6121f84c360a9854f4f8a7dbb2f46141 v0.1.0 (/home/ubuntu/.rustdef/rustdef_cell_061582ff6121f84c360a9854f4f8a7dbb2f46141)
    Finished dev [unoptimized + debuginfo] target(s) in 14.96s======>  ] 47/48
📦 Built wheel for CPython 3.6m to /home/ubuntu/.rustdef/target/wheels/rustdef_cell_061582ff6121f84c360a9854f4f8a7dbb2f46141-0.1.0-cp36-cp36m-manylinux2014_aarch64.whl
ignore /home/ubuntu/.rustdef/target/wheels/rustdef_cell_061582ff6121f84c360a9854f4f8a7dbb2f46141-0.1.0-cp38-cp38-manylinux2014_aarch64.whl


In [5]:
import numpy as np

In [6]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

array([ 4.7 , 18.58,  4.96])

In [7]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// mutable example
#[pyfn(m, "mult_inline")]
fn mult_inline<'py>(_py: Python<'py>, a: f64, x: &'py PyArrayDyn<f64>) -> PyResult<()> {
    let mut x = unsafe { x.as_array_mut() };
    x *= a;
    Ok(())
}

Use previous build


In [8]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.44297504, 0.68001904, 0.47378076, 0.85878938, 0.59299624,
        0.14037105, 0.62372361, 0.86531054, 0.92192754, 0.26999568]),
 array([0.64795717, 0.92139967, 0.04663736, 0.85287107, 0.04699888,
        0.24408926, 0.49788451, 0.07708571, 0.01114177, 0.488855  ]))

In [9]:
axpy(3.0, x, y)

array([1.9768823 , 2.96145678, 1.46797964, 3.42923921, 1.82598761,
       0.66520242, 2.36905534, 2.67301733, 2.77692438, 1.29884204])

In [10]:
mult_inline(12.0, x)

In [11]:
x

array([ 5.31570052,  8.16022845,  5.6853691 , 10.30547255,  7.11595489,
        1.68445263,  7.48468331, 10.38372651, 11.06313043,  3.23994813])